In [2]:
from imutils import paths 
from shutil import copyfile
import pandas as pd
import numpy as np
import cv2
import xml.etree.ElementTree as ET
import csv
import os

In [3]:
def make_dir(dirName):
    # Create a target directory & all intermediate 
    # directories if they don't exists
    if not os.path.exists(dirName):
        os.makedirs(dirName, exist_ok = True)
        print("[INFO] Directory " , dirName ,  " created")
    else:
        print("[INFO] Directory " , dirName ,  " already exists")

In [ ]:
SOURCE_PATH = 'dataset/facial_data'
TARGET_PATH = 'dataset/YOLOv5'

TRAIN_IMG_DIR = 'train/images'
TRAIN_LAB_DIR = 'train/labels'
make_dir(os.path.join(TARGET_PATH, TRAIN_IMG_DIR))
make_dir(os.path.join(TARGET_PATH, TRAIN_LAB_DIR))

TEST_IMG_DIR = 'test/images'
TEST_LAB_DIR = 'test/labels'
make_dir(os.path.join(TARGET_PATH, TEST_IMG_DIR))
make_dir(os.path.join(TARGET_PATH, TEST_LAB_DIR))

VAL_IMG_DIR = 'val/images'
VAL_LAB_DIR = 'val/labels'
make_dir(os.path.join(TARGET_PATH, VAL_IMG_DIR))
make_dir(os.path.join(TARGET_PATH, VAL_LAB_DIR))

In [5]:
def toCSV(source_path, set_='train', env='facial_data'):
    xml_path = os.path.join(source_path, set_, '{}.xml'.format(env))
    csv_file = os.path.join(source_path, set_, '{}.csv'.format(env))
    
    # Parse the XML file
    tree = ET.parse(xml_path)
    root = tree.getroot()

    # Open the CSV file for writing
    with open(csv_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        
        # Write the header row
        header = ["image", "xs", "ys", "xe", "ye"] + [f"p{num}x" for num in range(1,55)] + [f"p{num}y" for num in range(1,55)]
        writer.writerow(header)
        
        # Iterate over image elements in XML
        for image in root.findall(".//image"):
            # Filter for images in the "images/" directory
            image_file = image.get("file")
            if image_file.startswith("images/"):
                row = [image_file.split('/')[1]]
                
                # Extract box information
                box = image.find("box")
                if box is not None:
                    xs = int(box.get("left"))
                    ys = int(box.get("top"))
                    xe = xs + int(box.get("width"))
                    ye = ys + int(box.get("height"))
                    
                    # Add box coordinates to the row
                    row.extend([xs, ys, xe, ye])
                    
                    # Add part coordinates (assuming names 00 to 53 sequentially)
                    parts_x = []
                    parts_y = []
                    for part in sorted(box.findall("part"), key=lambda p: int(p.get("name"))):
                        parts_x.append(part.get("x"))
                        parts_y.append(part.get("y"))
                    
                    # Extend the row with part coordinates
                    row.extend(parts_x + parts_y)
                    
                    # Write row to CSV
                    writer.writerow(row)

In [6]:
toCSV(SOURCE_PATH, set_='test')

In [7]:
toCSV(SOURCE_PATH, set_='train')

In [8]:
toCSV(SOURCE_PATH, set_='val')

In [10]:
def convert(source_path, target_path, set_='train', env='facial_data'):
    img_path = os.path.join(source_path, set_, 'images')
    csv_file = os.path.join(source_path, set_, '{}.csv'.format(env))
    
    # import labels as a numpy array
    labels = pd.read_csv(csv_file,header=0).to_numpy()

    # initialize iterators
    iter1 = 0 
    iter2 = 0

    # init face counter
    face_count = 0

    # loop over the rows
    while iter1 < len(labels):
        # load the image
        imageName = labels[iter1][0]
        
        source_img = os.path.join(img_path, imageName)
        image = cv2.imread(source_img)
        H, W = image.shape[:2]
        
        target_img = os.path.join(target_path, set_, 'images', imageName)
        target_txt = open(os.path.join(target_path, set_,
                                'labels/{}.txt'.format(imageName.split('.png')[0])), "w")
        
        copyfile(source_img, target_img)
        
        # loop over the labels of this image
        while labels[iter1][0] == labels[iter2][0]:
            # extract coordinates of the bounding box
            xs, ys, xe, ye = labels[iter2][1:5]
            
            # extract 54 facial landmarks (108 values for x and y coordinates)
            landmarks = labels[iter2][5:5 + 108]
            
            # format bounding box and landmark values into a string
            label = 'face {} {} {} {}\n'.format(xs, ys, xe - xs, ye - ys)
            w = xe - xs
            h = ye - ys
                        
            xc = xs + w / 2
            yc = ys + h / 2
            
            # Format the label line with normalized bounding box and landmarks
            label = '0 {:.3f} {:.3f} {:.3f} {:.3f}'.format(xc / W, yc / H, w / W, h / H)
            
            # Normalize each landmark coordinate and add to the label
            for i in range(0, 108, 2):  # Iterate over landmarks by pairs (x, y)
                px = landmarks[i] / W
                py = landmarks[i + 1] / H
                label += ' {:.3f} {:.3f}'.format(px, py)
                
            label += '\n'
            
            # Write label to the file
            target_txt.write(label)
            face_count += 1
                        
            iter2 += 1
                        
            # break the loop if we exceeded the last row
            if iter2 == len(labels):
                break
                
        # equalize iterators
        iter1 = iter2


    print(iter2, face_count)

In [ ]:
convert(SOURCE_PATH, TARGET_PATH, set_='test')

In [ ]:
convert(SOURCE_PATH, TARGET_PATH, set_='train')

In [ ]:
convert(SOURCE_PATH, TARGET_PATH, set_='val')